## td_Qobj "advanced" methods

Feature of td_Qobj made for solver.


In [1]:
import qutip as qt
import numpy as np
from qutip import td_qobj
%load_ext cython

In [2]:
N = 5
destroy, create, Id = qt.destroy(N), qt.create(N), qt.qeye(N)
def exp_i(t,args):
    return np.exp(-1j*t)
def cos_w(t,args):
    return np.cos(args["w"]*t)
tlist = np.linspace(0,10,10000)
tlistlog = np.logspace(-3,1,10000)

# state vector as np array
vec = np.arange(N)*.5+.5j
vec_super = np.arange(N**2)*.5+.5j

# Construct td_Qobj of all type
td_cte1 = td_qobj.td_Qobj(Id)
td_cte2 = td_qobj.td_Qobj([Id])

td_func = td_qobj.td_Qobj([Id,[create,exp_i],[destroy,cos_w]],args={"w":2})
td_str = td_qobj.td_Qobj([Id,[create,"exp(-1j*t)"],[destroy,"cos(w*t)"]],args={'w':2.})
td_array = td_qobj.td_Qobj([Id,[create,np.exp(-1j*tlist)],[destroy,np.cos(2*tlist)]],tlist=tlist)
td_array_log = td_qobj.td_Qobj([Id,[create,np.exp(-1j*tlistlog)],[destroy,np.cos(2*tlistlog)]],tlist=tlistlog)

td_super = qt.liouvillian(td_func, c_ops=td_cte1)

## Product and expectation value

td_Qobj.rhs(t,state) = spmv(td_Qobj(t), state)  
td_Qobj.expect(t, state, real) = cy_expect_psi/cy_expect_rho_vec  (td_Qobj(t), state, real)

In [3]:
from qutip.cy.spmatfuncs import spmv, cy_expect_rho_vec, cy_expect_psi

In [4]:
spmv(td_func(2).data, vec) == td_func.rhs(2,vec)

array([ True,  True,  True,  True,  True], dtype=bool)

In [5]:
cy_expect_psi(td_str(2).data, vec, 0) == td_str.expect(2, vec, 0)

True

In [6]:
cy_expect_rho_vec(td_super(2).data, vec_super, 0) == td_super.expect(2, vec_super, 0)

True

## Function with state

For the solvers which have the option "with_state"

the td_Qobj can take coefficient function with the signature:

**def coeff(t, psi. args)**

and must be called with
td_Qobj.with_state(self, t, psi, args={}, data=False)

**The normal call, rhs and expect will create and error.**
**Mixing with normal function coefficient will cause error.**
Mixing with string or array will work for now.

In [7]:
def coeff_state(t, psi, args):
    return np.max(psi)*args["w"]
td_state = td_qobj.td_Qobj([Id, [destroy, coeff_state]],args={"w":1})

In [8]:
td_state.with_state(2,vec)

Quantum object: dims = [[5], [5]], shape = (5, 5), type = oper, isherm = False
Qobj data =
[[ 1.00000000+0.j          2.00000000+0.5j         0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j        ]
 [ 0.00000000+0.j          1.00000000+0.j          2.82842712+0.70710678j
   0.00000000+0.j          0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000+0.j          1.00000000+0.j
   3.46410162+0.8660254j   0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   1.00000000+0.j          4.00000000+1.j        ]
 [ 0.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          1.00000000+0.j        ]]

## Cython object and "Compiling"

There is a cython version of the td_qobj for fast call:  
- qutip.cy.td_qobj_cy.cy_qobj  
- qutip.cy.td_qobj_cy.cy_cte_qobj  
- qutip.cy.td_qobj_cy.cy_td_qobj  

The cython is created when the "compile" method is created.
The cython object contain fast version of the call, expect and rhs (spmv) methods.

In [9]:
td_str.compiled = False
print("Before compilation")
%timeit td_str(2, data=True)
%timeit td_str.rhs(2,vec)
%timeit td_str.expect(2,vec,0)
td_str.compile()
print("After compilation")
%timeit td_str(2, data=True)
%timeit td_str.rhs(2,vec)
%timeit td_str.expect(2,vec,0)

Before compilation
132 µs ± 611 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
145 µs ± 307 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
148 µs ± 7.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
After compilation
14.7 µs ± 201 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
9.02 µs ± 48.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
9.84 µs ± 166 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Remove intermediary compilation

When a string based td_qobj is created, each one of the strings coefficients are compiled to functions.
When it is "compiled", then the string coefficient are recompiled so they all fit in one function called from cython.

My expected workflow is   
- Create the td_qobjs 
- Do the math( \*, +, dag, ...)
- compile
- Use  

Then the first compilation is a waste of time. So at the creation, there is the option not to compile the strings coefficient:  
td_Qobj(..., raw_str=True)


In [10]:
td_o = td_qobj.td_Qobj([Id,[create,"sin(t)"],[destroy,"cos(w*t)"]], args={'w':2.}, raw_str=True)
print(td_o(1.))
td_o.compile()
print(td_o(1.))

Quantum object: dims = [[5], [5]], shape = (5, 5), type = oper, isherm = False
Qobj data =
[[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
Quantum object: dims = [[5], [5]], shape = (5, 5), type = oper, isherm = False
Qobj data =
[[ 1.         -0.41614684  0.          0.          0.        ]
 [ 0.84147098  1.         -0.5885205   0.          0.        ]
 [ 0.          1.19001968  1.         -0.72078746  0.        ]
 [ 0.          0.          1.4574705   1.         -0.83229367]
 [ 0.          0.          0.          1.68294197  1.        ]]


## apply
Pass a function ( Qobj, \*args, \*\*kwargs) -> Qobj to act on each component of the td_qobj.

Will only be mathematicaly valid if the transformation is linear.

In [11]:
def multiply(qobj,b,factor = 3.):
    return qobj*b*factor

print(td_func.apply(multiply,2)(2) == td_func(2)*6)
print(td_func.apply(multiply,2,factor=2)(2) == td_func(2)*4)

True
True


## apply_decorator
Transform the functions containing the time dependence using a decorator.

The decorator must return a function of (t, **kwargs).

Do not modify the constant part (the contant part do not have a function f(t) = 1).


In [12]:
def rescale_time_and_scale(f_original, time_scale, factor=2.):
    def f(t, *args, **kwargs):
        return f_original(time_scale*t, *args, **kwargs)*factor
    return f

print(td_func.apply_decorator(rescale_time_and_scale,2)(2) == td_func(4)*2-Id)
print(td_func.apply_decorator(rescale_time_and_scale,3,factor=3)(2) == 
      td_func(6)*3.0 - 2*Id)

True
True


td_Qobj based on string and np.array are changed to a function then the decorator is applied. There are option so that the type of coefficient stay unchanged: 

    str_mod : change the string -> str_mod[0] + str + str_mod[1]
    
    inplace_np : modify the array  (array[i] = decorator(lambda v: v)(array[i]))
         *any modification that rescale the time will not work properly
        
Decorator can cause problem when used in parallel. (function cannot be pickled error)

In [13]:
td_func_1 = td_qobj.td_Qobj([[create,exp_i]],args={"w":2})
td_str_1 = td_qobj.td_Qobj([[create,"exp(-1j*t)"]],args={'w':2.})
td_array_1 = td_qobj.td_Qobj([[create,np.exp(-1j*tlist)]],tlist=tlist)

def square_qobj(qobj):
    return qobj*qobj

def square_f(f_original):
    def f(t, *args, **kwargs):
        return f_original(t, *args, **kwargs)**2
    return f

t1 = td_func_1.apply(square_qobj).apply_decorator(square_f)
print(t1(2) == td_func_1(2)*td_func_1(2))
print((t1.ops[0][2]))

t1 = td_str_1.apply(square_qobj).apply_decorator(square_f)
print(t1(2) == td_str_1(2)*td_str_1(2))
print("str not updated:", (t1.ops[0][2]))

t1 = td_str_1.apply(square_qobj).apply_decorator(square_f, str_mod=["(",")**2"])
print(t1(2) == td_str_1(2)*td_str_1(2))
print("str  updated:",(t1.ops[0][2]))

t1 = td_array_1.apply(square_qobj).apply_decorator(square_f)
print(t1(2) == td_array_1(2)*td_array_1(2))
print("array not updated:",(t1.ops[0][2]))

t1 = td_array_1.apply(square_qobj).apply_decorator(square_f, inplace_np=1)
print(t1(2) == td_array_1(2)*td_array_1(2))
print("array  updated:",(t1.ops[0][2]))

True
<function square_f.<locals>.f at 0x7fa5c7a97730>
True
str not updated: exp(-1j*t)
True
str  updated: (exp(-1j*t))**2
True
array not updated: [ 1.00000000-0.j          0.99999950-0.0010001j   0.99999800-0.0020002j
 ..., -0.84015800+0.54234171j -0.83961518+0.54318168j
 -0.83907153+0.54402111j]
True
array  updated: [ 1.00000000-0.j          0.99999800-0.0020002j   0.99999200-0.00400039j
 ...,  0.41173093-0.91130546j  0.40990732-0.91212718j
  0.40808206-0.91294525j]


## Removing redundant Qobj

When multiple components of the td_Qobj are made from the same Qobj, you can unite them with the "compress" method.
It is only done with the same form of time dependance:


In [14]:
small = qt.destroy(2)
def f1(t,args):
    return np.sin(t)
def f2(t,args):
    return np.cos(args["w"]*t)
def f3(t,args):
    return np.sin(args["w"]*t)
def f4(t,args):
    return np.cos(t)

In [15]:
td_redoundance = td_qobj.td_Qobj([qt.qeye(2),[small,"sin(t)"],[small,"cos(w*t)"],[small,"sin(w*t)"],
                                  [small,"cos(t)"]],args={'w':2.})
td_redoundance1 = td_qobj.td_Qobj([qt.qeye(2),[small,"sin(t)"],[small,"cos(w*t)"],[small,"sin(w*t)"],
                                   [small,"cos(t)"]],args={'w':2.})
td_redoundance2 = td_qobj.td_Qobj([qt.qeye(2),[small,f1],[small,f2],[small,f3],[small,f4]],args={'w':2.})
td_redoundance3 = td_qobj.td_Qobj([qt.qeye(2),[small,np.sin(tlist)],[small,np.cos(2*tlist)],
                                   [small,np.sin(2*tlist)],[small,np.cos(tlist)]],tlist=tlist)
td_redoundance4 = td_qobj.td_Qobj([qt.qeye(2),[small,f1],[small,"cos(w*t)"],
                                   [small,np.sin(2*tlist)],[small,"cos(t)"]],args={'w':2.},tlist=tlist)

td_redoundance1.compress()
print(td_redoundance1.to_list())
print(len(td_redoundance1.ops))
print(td_redoundance(1.) == td_redoundance1(1.))
td_redoundance2.compress()
print(len(td_redoundance2.ops))
print(td_redoundance(1.) == td_redoundance2(1.))
td_redoundance3.compress()
print(len(td_redoundance3.ops))
print(td_redoundance(1.) == td_redoundance3(1.))
td_redoundance4.compress()
print(len(td_redoundance4.ops))
print(td_redoundance(1.) == td_redoundance4(1.))

[Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 1.  0.]
 [ 0.  1.]], [Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 0.  1.]
 [ 0.  0.]], '(((sin(t)) + (cos(w*t))) + (sin(w*t))) + (cos(t))']]
1
True
1
True
1
True
3
True


## Compilation and arguments
When compiling, the local arguments of each part are lost.  

This may cause a problem since the behaviour is not the same than addition when not compiled.

Calling the method 'argument' update the arguments event after compilation (and do not need recompilation).

In [16]:
td_args_1 = td_qobj.td_Qobj([Id, [destroy, "cos(w*t)"]],args={'w':1.})
td_args_2 = td_qobj.td_Qobj([Id, [destroy, "cos(w*t)"]],args={'w':2.})
td_str_sum = td_args_1 + td_args_2
before_compile = td_str_sum(2)
td_str_sum_compiled_call = td_str_sum.get_compiled_call()
print(before_compile == td_str_sum_compiled_call(2.))
# Once comiled, the call use the compiled function
# Which args is used once compiled is not fixed. ()
print(td_str_sum(2) == td_str_sum_compiled_call(2.)) 
td_str_sum.arguments({'w':3})
td_str_sum_compiled_call = td_str_sum.get_compiled_call()
td_args_3 = td_qobj.td_Qobj([Id, [destroy, "cos(w*t)"]],args={'w':3.})*2
print(td_args_3(2.) == td_str_sum_compiled_call(2.))

False
True
True


In [17]:
td_args_1 = td_qobj.td_Qobj([Id, [destroy, cos_w]],args={'w':1.})
td_args_2 = td_qobj.td_Qobj([Id, [destroy, cos_w]],args={'w':2.})
td_str_sum = td_args_1 + td_args_2
before_compile = td_str_sum(2)
td_str_sum_compiled_call = td_str_sum.get_compiled_call()
print(before_compile == td_str_sum_compiled_call(2.))
# Once comiled, the call use the compiled function
# Which args is used once compiled is not fixed. ()
print(td_str_sum(2) == td_str_sum_compiled_call(2.)) 
td_str_sum.arguments({'w':3})
td_str_sum_compiled_call = td_str_sum.get_compiled_call()
td_args_3 = td_qobj.td_Qobj([Id, [destroy, "cos(w*t)"]],args={'w':3.})*2
print(td_args_3(2.) == td_str_sum_compiled_call(2.))

False
True
True


## cimport cython object

The cython object can be 'cimported'.
```
There are 3 objects:
    cy_td_qobj  : time dependent cases
    cy_cte_qobj : constant cases
    cy_qobj     : general objects, both others cases inherit from it
```

```
cdef class cy_qobj:
    cdef void _rhs_mat(self, double t, complex* vec, complex* out)
    cdef complex _expect_mat(self, double t, complex* vec, int isherm)
    cdef complex _expect_mat_super(self, double t, complex* vec, int isherm)
```

In [18]:
%%cython
from qutip.cy.td_qobj_cy cimport cy_td_qobj, cy_cte_qobj, cy_qobj
cimport numpy as np


def rhs_call_from_cy(cy_qobj qobj, double t, np.ndarray[complex, ndim=1] vec, np.ndarray[complex, ndim=1] out):
    qobj._rhs_mat(t,&vec[0],&out[0])
    
    
def expect_call_from_cy(cy_qobj qobj, double t, np.ndarray[complex, ndim=1] vec, int isherm):
    return qobj._expect_mat(t,&vec[0],isherm)
    
    
def rhs_cdef_timing(cy_qobj qobj, double t, np.ndarray[complex, ndim=1] vec, np.ndarray[complex, ndim=1] out):
    cdef int i
    for i in range(10000):
        qobj._rhs_mat(t,&vec[0],&out[0])

        
def expect_cdef_timing(cy_qobj qobj, double t, np.ndarray[complex, ndim=1] vec, int isherm):
    cdef complex aa = 0.
    cdef int i
    for i in range(10000):
        aa = qobj._expect_mat(t, &vec[0], isherm)
    return aa

def rhs_def_timing(qobj, double t, np.ndarray[complex, ndim=1] vec, complex[::1] out):
    cdef int i
    for i in range(10000):
        out = qobj.rhs(t,vec)
        
def expect_def_timing(qobj, double t, np.ndarray[complex, ndim=1] vec, int isherm):
    cdef complex aa = 0.
    cdef int i
    for i in range(10000):
        aa = qobj.expect(t, vec, isherm)
    return aa


In [19]:
td_str.compile()
print(expect_call_from_cy(td_str.compiled_Qobj, 2, vec, 0) - td_str.expect(2,vec,0))
%timeit expect_def_timing(td_str.compiled_Qobj, 2, vec, 0)
%timeit expect_cdef_timing(td_str.compiled_Qobj, 2, vec, 0)

0j
61.6 ms ± 1.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
49.7 ms ± 236 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
out = np.zeros(N,dtype=np.complex128)
rhs_call_from_cy(td_str.compiled_Qobj, 2, vec, out)
print( [a - b for a,b in zip(out, td_str.rhs(2,vec))])
%timeit rhs_def_timing(td_str.compiled_Qobj, 2, vec, out)
%timeit rhs_cdef_timing(td_str.compiled_Qobj, 2, vec, out)

[0j, 0j, 0j, 0j, 0j]
65.1 ms ± 390 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
25.3 ms ± 96.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
td_cte = td_qobj.td_Qobj([Id])
td_cte.compile()
out = np.zeros(N,dtype=np.complex128)
rhs_call_from_cy(td_cte.compiled_Qobj, 2, vec, out)
print( [a - b for a,b in zip(out, td_cte.rhs(2,vec))])
%timeit rhs_def_timing(td_cte.compiled_Qobj, 2, vec, out)
%timeit rhs_cdef_timing(td_cte.compiled_Qobj, 2, vec, out)

[0j, 0j, 0j, 0j, 0j]
38.1 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
207 µs ± 2.83 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Compiled string code



In [22]:
print(td_str.compile(code=True))


# This file is generated automatically by QuTiP.

import numpy as np
cimport numpy as np
cimport cython
np.import_array()
cdef extern from "numpy/arrayobject.h" nogil:
    void PyDataMem_NEW_ZEROED(size_t size, size_t elsize)
    void PyArray_ENABLEFLAGS(np.ndarray arr, int flags)
from qutip.cy.spmatfuncs cimport spmvpy
from qutip.cy.inter cimport spline_complex_t_second, spline_complex_cte_second
from qutip.cy.inter cimport spline_float_t_second, spline_float_cte_second
from qutip.cy.math cimport erf
cdef double pi = 3.14159265358979323

cdef extern from "Python.h":
    object PyLong_FromVoidPtr(void *)
    void* PyLong_AsVoidPtr(object)

include '/home/eric/anaconda3/lib/python3.6/site-packages/qutip-4.3.0.dev0+ec9b31a-py3.6-linux-x86_64.egg/qutip/cy/complex_math.pxi'

cdef class coeff_args:
    cdef double w

    def set_array_imag(self, int N, double[::1] tlist, complex[::1] array, complex[::1] spline):
        if N == -1:
            pass
        else:
            raise Exception